In [2]:
import os
import pickle
import numpy as np
import pandas as pd

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dropout, Dense, Flatten
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import TimeSeriesSplit

In [4]:
from tensorflow.keras.layers import LSTM, Dropout

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping


merged_data = pd.read_csv(r"3_labels.csv")


In [6]:
X = merged_data.iloc[:, :-1].values
y = merged_data.iloc[:, -1].values

X = X.reshape(X.shape[0], X.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
import gc
del merged_data,X,y
gc.collect()

266

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
model = Sequential()

# CNN layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())

# # LSTM layers with dropout
# model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dropout(0.3))

# Fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('cnn_lstm_wesad_ablation.keras', monitor='val_accuracy', save_best_only=True, mode='max')





In [9]:
history = model.fit(X_train, y_train, epochs=20, batch_size=1024,validation_data=(X_test, y_test), callbacks=[early_stopping, model_checkpoint])

Epoch 1/20
32101/32101 [==============================] - 111s 3ms/step - loss: 0.2807 - accuracy: 0.8863 - val_loss: 0.1845 - val_accuracy: 0.9304
Epoch 2/20
32101/32101 [==============================] - 111s 3ms/step - loss: 0.2034 - accuracy: 0.9221 - val_loss: 0.1853 - val_accuracy: 0.9285
Epoch 3/20
32101/32101 [==============================] - 118s 4ms/step - loss: 0.1850 - accuracy: 0.9297 - val_loss: 0.1552 - val_accuracy: 0.9423
Epoch 4/20
32101/32101 [==============================] - 115s 4ms/step - loss: 0.1743 - accuracy: 0.9340 - val_loss: 0.1326 - val_accuracy: 0.9525
Epoch 5/20
32101/32101 [==============================] - 116s 4ms/step - loss: 0.1665 - accuracy: 0.9371 - val_loss: 0.1518 - val_accuracy: 0.9421
Epoch 6/20
32101/32101 [==============================] - 116s 4ms/step - loss: 0.1606 - accuracy: 0.9393 - val_loss: 0.1309 - val_accuracy: 0.9518
Epoch 7/20
32101/32101 [==============================] - 116s 4ms/step - loss: 0.1552 - accuracy: 0.9415 - val_

In [10]:
import gc
del X_train
gc.collect()

1629

In [11]:
from sklearn.metrics import classification_report
predict_x=model.predict(X_test)



440239/440239 [==============================] - 439s 996us/step


In [12]:
import gc
del X_test,y_train
gc.collect()

688

In [13]:
y_pred=np.argmax(predict_x,axis=1)
print(f"Classification Report: \n{classification_report(y_test, y_pred)}")
from sklearn.metrics import roc_curve, auc, roc_auc_score

Classification Report: 
              precision    recall  f1-score   support

         0.0       0.98      0.96      0.97   8298360
         1.0       0.97      0.99      0.98   3697260
         2.0       0.90      0.93      0.92   2092020

    accuracy                           0.97  14087640
   macro avg       0.95      0.96      0.96  14087640
weighted avg       0.97      0.97      0.97  14087640



In [14]:

# print(f"Confusion Matrix for:\n")
# def plot_confusion_matrix(y_true, y_pred, classes):
#   cm = confusion_matrix(y_true, y_pred)
#   plt.figure(figsize=(8, 6))
#   sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
#   plt.title("Confusion Matrix")
#   plt.xlabel("Predicted")
#   plt.ylabel("True")
#   plt.show()

# # Plot the confusion matrix
# y_pred_prob = predict_x
# y_pred = np.argmax(y_pred_prob, axis=1)
# plot_confusion_matrix(y_test, y_pred, classes=['0', '1', '2'])
# # Compute ROC curve and ROC area for each class
# print(f"ROC for :\n")
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import roc_curve, auc
# from sklearn.preprocessing import label_binarize
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import roc_auc_score
# n_classes = 3
# # Binarize the labels
# y_test_bin = label_binarize(y_test, classes=np.arange(n_classes))
# y_pred_bin = label_binarize(y_pred, classes=np.arange(n_classes))
# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()

# for i in range(n_classes):
#   fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_bin[:, i])
#   roc_auc[i] = auc(fpr[i], tpr[i])

# # Plotting ROC curves
# plt.figure(figsize=(10, 8))

# # Plot ROC curve for each class
# for i in range(n_classes):
#   plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

# plt.plot([0, 1], [0, 1], 'k--', lw=2)  # Diagonal line for reference
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc="lower right")
# plt.show()

# print(f"Training and Validation Curves")
# # Plot training & validation accuracy values
# plt.figure(figsize=(12, 6))
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

# # Plot training & validation loss values
# plt.figure(figsize=(12, 6))
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()

In [15]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9661610461368974
